In [ ]:
# Run in python 3.7+
# Script for all sensor data

import sys
import arable
from   arable.client   import ArableClient as a
a = a()
import re
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from   datetime        import timedelta
from   datetime        import datetime
import pandas          as     pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from   io              import StringIO
from   sensor_includes import email, password_ccber, tenant_ccber
#import arable_data_dict as dd

# Datetime object to format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverse dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temp lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)
    
a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

dt = datetime.now()

device_ids     = [] # Ex. A00****
device_names   = [] # Ex. Lagoon Ice Plants
device_created = [] # Ex. 2019-10-10T00:00:00

# The devices of interest have Z in the Location Name on the Arable website.
# These Location Names are added to device_names.
# The corresponding Device ID is added to device_ids.
for i in a.devices():
    if i['location']['name'] != '':
        device_names.append(str(i['location']['name']))
        device_ids.append(str(i['name']))
        device_created.append(str(i['created']))

print(['{:<7}'.format(str(i)) for i in device_ids])
print(device_names)
print(device_created)

# Converts each item in device_ids to a list.          
device_ids = list(map(lambda el:[el], device_ids))

def arable_query(s, b, c, d, e, f, g):
    df = a.query(select = str(s),
               format = str(b),
               devices = c,
               measure = str(d),
               order = str(e),
               end = f,
               start = g,
               limit = 100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df

lat = []
lon = []
for i in device_ids:
    df = arable_query('all', 'csv', i, "daily", "time", dt_to_ymd_hms(dt), dt_to_ymd_hms(dt - timedelta(days=1)))
    lat.append(float(str(df['lat'].values.astype(float)).strip('[ ]')))
    lon.append(float(str(df['long'].values.astype(float)).strip('[ ]')))

data_location = {'name': device_ids, 'location': device_names, 'lat': lat, 'lon': lon}
df_location   = pd.DataFrame(data=data_location)

# Monthly
m_start    = []
m_start_fn = []
m_end      = []
m_end_fn   = []

# Yearly
y_start    = []
y_start_fn = []
y_end      = []
y_end_fn   = []

# All Time
astart     = []
start_fn   = []
end        = []
end_fn     = []

for j in range(len(device_created)):
    ms  = []
    msf = []
    me  = []
    mef = []
    
    ys  = []
    ysf = []
    ye  = []
    yef = []
    
    s   = []
    sf  = []
    e   = []
    ef  = []
    
    def new_month(new_m):
        append_ymd_hms(ms, new_m)
        append_string(me, new_m)
        append_ym(msf, new_m)  
        append_ym(mef, new_m)
        
    def new_year(new_y):
        append_ymd_hms(ys, new_y)
        append_string(ye, new_y)
        append_ym(ysf, new_y)
        append_ym(yef, new_y)
    
    device_created_ymd = (device_created[j])[:10]
    start              = ymd_to_dt(device_created_ymd)
    append_string(ms, start)
    append_string(ys, start)
    append_string(s, start)
    append_ym(msf, start)
    append_ym(ysf, start)
    append_ym(sf, start)
    
    start  = start.replace(day=1)
    
    if start.year == dt.year:
        for i in range(start.month+1, dt.month+1):
            new_month(start.replace(month=i))
    
    elif start.year != dt.year: 
        for i in range(dt.year-start.year):
            for j in range(start.month+1, 13): 
                new_month(start.replace(month=j))
        for i in range(start.year+1, dt.year+1):
            newyr = start.replace(year=i, month=1)
            new_year(newyr)
            for j in range(1, dt.month+1):
                new_month(newyr.replace(month=j))
    
    append_string(me, dt)
    append_string(ye, dt)
    append_string(e, dt)
    append_ym(mef, dt)
    append_ym(yef, dt)
    append_ym(ef, dt)
    
    append_temp_list(ms, m_start)
    append_temp_list(me, m_end)
    append_temp_list(msf, m_start_fn)
    append_temp_list(mef, m_end_fn)
    
    append_temp_list(ys, y_start)
    append_temp_list(ye, y_end)
    append_temp_list(ysf, y_start_fn)
    append_temp_list(yef, y_end_fn)
    
    append_temp_list(s, astart)
    append_temp_list(e, end)
    append_temp_list(sf, start_fn)
    append_temp_list(ef, end_fn)
    
# Writes a .csv file for each device.
# Hourly data is separated by month; daily data is separated by year.
# Ex. A000***_hourly_2019-01_2019-02.csv
import time
def write_csv_file(sta, end, i, sta_f, end_f, device): 
    print(str(device)[2:-2] + ' has been active since ' + str(sta[0]) + '.')
    for j in range(len(sta)):
        fn = str(device)[2:-2] + '_' + str(i) + '_' + str(sta_f[j]) + '_' + str(end_f[j]) + '.csv'
        try:
            df = a.query(select = 'all', format = 'csv', devices = device, measure = str(i), order = "time", end = end[j], start = sta[j], limit = 100000000)
            df = StringIO(df)
            df = pd.read_csv(df, sep=',', error_bad_lines=False)
            df = df.drop('location', 1)
            df.to_csv(fn, sep = ",")
            print('Successfully wrote ' + fn)
        except:
            print('Error writing ' + fn)
            continue   
def inputs():
    # Writes data to a .csv file
f_input = raw_input('Write data to a .csv file? (y/n) ')
if f_input == 'y':
    print('\n Select data type: ')
    print(str('{:<2d}'.format(1) + '{:<8}'.format('Daily')      ))
    print(str('{:<2d}'.format(2) + '{:<8}'.format('Hourly')     ))
    print(str('{:<2d}'.format(3) + '{:<8}'.format('Soil')       ))
    print(str('{:<2d}'.format(4) + '{:<8}'.format('Calibrated') ))
    d_input = int(raw_input('[1-4]  '))
    
    print('\n Select timeframe:  ')
    print(str('{:<2d}'.format(1) + '{:<20}'.format('Year')))
    print(str('{:<2d}'.format(2) + '{:<20}'.format('Month')))
    print(str('{:<2d}'.format(3) + '{:<20}'.format('All Time')))
    t_input = int(raw_input('[1-3]  '))
    
    all_q = raw_input('\n All devices? (y/n) ')
    if all_q == 'y':
        for i in range(len(device_ids)):
            if t_input == 1:
                if d_input == 1:
                    write_csv_file(y_start[i], y_end[i],      "daily", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(y_start[i], y_end[i],     "hourly", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(y_start[i], y_end[i],    "aux_raw", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(y_start[i], y_end[i], "calibrated", y_start_fn[i], y_end_fn[i], device_ids[i])
            if t_input == 2:
                if d_input == 1:
                    write_csv_file(m_start[i], m_end[i],      "daily", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(m_start[i], m_end[i],     "hourly", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(m_start[i], m_end[i],    "aux_raw", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(m_start[i], m_end[i], "calibrated", m_start_fn[i], m_end_fn[i], device_ids[i])
            if t_input == 3:
                if d_input == 1:
                    write_csv_file(astart[i], end[i],      "daily", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(astart[i], end[i],     "hourly", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(astart[i], end[i],    "aux_raw", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(astart[i], end[i], "calibrated", start_fn[i], end_fn[i], device_ids[i])
                    
                    
    elif all_q == 'n':
        print('\n Select device: ')
        print('-----------------------------------')
        print(str('{:6}'.format('Number')) + '  ' + '{:<7}'.format('ID') + '  ' + '{:<10}'.format('Location Name'))
        print('-----------------------------------')
        for i in range(len(device_ids)):
            print(str('{:<6d}'.format(i+1)) + '  ' + '{:^7}'.format(str(device_ids[i]).strip("[' ']")) + '  ' + '{:^10}'.format(str(device_names[i]).strip("['Z ']")))
            print('-----------------------------------')
        j = int(raw_input('[1-' + str(len(device_ids)) + ']  '))
        if 1 <= j <= int(len(device_ids)):
            i = j-1
            if t_input == 1:
                if d_input == 1:
                    write_csv_file(y_start[i], y_end[i],      "daily", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(y_start[i], y_end[i],     "hourly", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(y_start[i], y_end[i],    "aux_raw", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(y_start[i], y_end[i], "calibrated", y_start_fn[i], y_end_fn[i], device_ids[i])
            if t_input == 2:
                if d_input == 1:
                    write_csv_file(m_start[i], m_end[i],      "daily", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(m_start[i], m_end[i],     "hourly", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(m_start[i], m_end[i],    "aux_raw", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(m_start[i], m_end[i], "calibrated", m_start_fn[i], m_end_fn[i], device_ids[i])
            if t_input == 3:
                if d_input == 1:
                    write_csv_file(astart[i], end[i],      "daily", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(astart[i], end[i],     "hourly", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(astart[i], end[i],    "aux_raw", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(astart[i], end[i], "calibrated", start_fn[i], end_fn[i], device_ids[i])
        else:
            print('User input error')
    
    else:
        print('User input error')
elif f_input == 'n':
    print('\n No files written.')
else:
    print('User input error')
    
from flask import Flask
app = Flask(__name__)
from flask import request
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        return do_the_login()
    else:
        return show_the_login_form()



In [49]:
import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber

a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

from   datetime          import timedelta
from   datetime          import datetime


import pandas            as     pd

dt = datetime.now()

from   io                import StringIO

from arable_data_dict import Hourly_Data_Columns, Daily_Data_Columns, Soil_Data_Columns

# Datetime object to string with format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append string with format "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverses dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to string with format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to string with format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append string with format "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temporary lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

def arable_query(s, b, c, d, e, f, g):
    df = a.query( select = str(s),
                  format = str(b),
                 devices = c,
                 measure = str(d),
                   order = str(e),
                     end = f,
                   start = g,
                   limit = 5000) #100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df

start_current_month = ym_to_dt(dt_to_ym(dt)).replace(day=1)
print(start_current_month)
print(dt_to_ym(start_current_month))

today = dt
print(today)

2020-04-01 00:00:00
2020-04
2020-04-03 16:35:45.578000


In [61]:
df = a.query(select = 'all', 
                     format = 'csv', 
                     devices = "['A000536']", 
                     measure = str("aux_raw"), 
                     order = "time", 
                     end = today, 
                     start = start_current_month, 
                     limit = 100000000)
df = StringIO(df)
df = pd.read_csv(df, sep=',', error_bad_lines=False)
df.keys()
raw_VWC = df['sdi12_value_0'].to_list()
df = df.dropna(axis='columns', thresh=2)#how='all')
df.insert(1, 'Habitat', 'Annual_Grass')
df.insert(1, 'Site', 'Campus_Lagoon')
tm = df['time'].to_list()
date_stamp = []
time_stamp = []
for i in range(len(tm)):
    date_stamp.append(str(tm[i])[:10])
    time_stamp.append(str(tm[i])[11:][:8])
df.insert(1, 'Time', time_stamp)
df.insert(1, 'Date', date_stamp) 

cal_VWC = []
for i in range(len(raw_VWC)):
    x = float(i)
    cal_VWC.append(float(4.3*10**(-6)*x**(3) - 5.5*10**(-4)*x**(2) + 2.92*10**(-2)*x - 5.3*10**(-2)))
df.insert(7, '5TE Calibrated VWC (m3/m3)', cal_VWC)
df = df.drop('location', axis='columns')
df = df.drop('time', axis='columns')
df = df.rename(mapper=Daily_Data_Columns, axis='columns')
df.head()


,Date,Time,Site,Habitat,device,5TE Calibrated VWC (m3/m3),sdi12_value_0,sdi12_value_1,sdi12_value_2,sdi12_version,sdi12_vendor_id,sdi12_sensor_model,sdi12_sensor_version,sdi12_sensor_sn,analog
0,2020-04-01,00:03:01,Campus_Lagoon,Annual_Grass,A000536,-0.053000,8.64,0.02,17.3,13,DECAGON,5TE,400,None,0.1364
1,2020-04-01,00:08:01,Campus_Lagoon,Annual_Grass,A000536,-0.024346,8.64,0.02,17.3,13,DECAGON,5TE,400,None,0.1364
2,2020-04-01,00:13:02,Campus_Lagoon,Annual_Grass,A000536,0.003234,8.64,0.02,17.4,13,DECAGON,5TE,400,None,0.1364
3,2020-04-01,00:18:03,Campus_Lagoon,Annual_Grass,A000536,0.029766,8.63,0.02,17.4,13,DECAGON,5TE,400,None,0.1364
4,2020-04-01,00:23:04,Campus_Lagoon,Annual_Grass,A000536,0.055275,8.63,0.02,17.4,13,DECAGON,5TE,400,None,0.1363


In [4]:
import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber

a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

from   datetime          import timedelta
from   datetime          import datetime


import pandas            as     pd

dt = datetime.now()

from   io                import StringIO

from arable_data_dict import Hourly_Data_Columns, Daily_Data_Columns, Soil_Data_Columns

Soil_Data_Columns = {
    "device"          :"Device ID",
    "time"            :"Time stamp",
    "sdi12_value_0"   :"5TE raw VWC (dielectric permittivity)",
    "sdi12_value_1"   :"5TE Bulk EC (dS/m)",
    "sdi12_value_2"   :"5TE Temp (C)",
    "analog"          :"EC-5 Sensor VWC (%)"
}

# Datetime object to string with format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append string with format "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverses dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to string with format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to string with format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append string with format "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temporary lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

def arable_query(s, b, c, d, e, f, g):
    df = a.query( select = str(s),
                  format = str(b),
                 devices = c,
                 measure = str(d),
                   order = str(e),
                     end = f,
                   start = g,
                   limit = 5000) #100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df

start_current_month = ym_to_dt(dt_to_ym(dt)).replace(day=1)
print(start_current_month)
print(dt_to_ym(start_current_month))

today = dt

import os
dirpath = 'C:/Users/ccber-arables/Box/'
#dirpath = 'C:/Users/15303/Documents/'

def write_csv_file(sta, end, data_type, data_type_fn, folder, cm, m, remove, dev, loc, site): 
    logfile = 'Log_' + str(dt_to_ymd(today)) + '.txt'
    devicee = "['" + str(dev) + "']"
    fn = str(folder) + str(loc) + '_' + str(cm) + '_' + str(data_type_fn) + '.csv'
    print(fn)
    try:
        df = a.query(select = 'all', 
                     format = 'csv', 
                     devices = devicee, 
                     measure = str(data_type), 
                     order = "time", 
                     end = end, 
                     start = sta, 
                     limit = 100000000)
        
        df = StringIO(df)
        df = pd.read_csv(df, sep=',', error_bad_lines=False)
        while data_type_fn == "Soil":
            raw_VWC = df['sdi12_value_0'].to_list()
            cal_VWC = []
            for i in range(len(raw_VWC)):
                x = float(raw_VWC[i])
                print(x)
                cal = float(4.3*10**(-6)*x**(3) - 5.5*10**(-4)*x**(2) + 2.92*10**(-2)*x - 5.3*10**(-2))
                print(cal)
                cal_VWC.append(cal)
            df = df.dropna(axis='columns', thresh=2)#how='all')
            tm = df['time'].to_list()
            date_stamp = []
            time_stamp = []
            for i in range(len(tm)):
                date_stamp.append(str(tm[i])[:10])
                time_stamp.append(str(tm[i])[11:][:8])
            df.insert(2, 'Time', time_stamp)
            df.insert(2, 'Date', date_stamp) 
            df.insert(2, 'Habitat', loc)
            df.insert(2, 'Site', site)
            for i in remove:
                print("Remove " + i)
                try:                
                    df = df.drop(i, axis='columns')
                except:
                    pass
            df = df.rename(mapper=m, axis='columns')
            df.insert(6, '5TE Calibrated VWC (m^3/m^3)', cal_VWC)
            df.to_csv(fn, sep = ",")
            print('Successfully wrote ' + fn)
            open(logfile, 'a+').write('Successfully wrote ' + fn + '\n')
        else:
            df = df.dropna(axis='columns', thresh=2)#how='all')
            tm = df['time'].to_list()
            date_stamp = []
            time_stamp = []
            for i in range(len(tm)):
                date_stamp.append(str(tm[i])[:10])
                time_stamp.append(str(tm[i])[11:][:8])
            df.insert(2, 'Time', time_stamp)
            df.insert(2, 'Date', date_stamp) 
            df.insert(2, 'Habitat', loc)
            df.insert(2, 'Site', site)
            for i in remove:
                print("Remove " + i)
                df = df.drop(i, axis='columns')
            df = df.rename(mapper=m, axis='columns')
            df.to_csv(fn, sep = ",")
            print('Successfully wrote ' + fn)
            open(logfile, 'a+').write('Successfully wrote ' + fn + '\n')
    except:
        open(logfile, 'a+').write('Failure writing ' + fn + '\n')
        #pass       


#----Site
#--------Habitat
#---------------Year
#-------------------Daily
#-------------------Hourly
#-------------------Soil
   
Sensor_ID            = ['A000318']

Site                 = ['Campus_Lagoon']
Habitat              = ['Coastal_Sage_Scrub']

Current_Year         = start_current_month.strftime("%Y")

Columns_to_Remove    = [['location','time','sdi12_version','sdi12_vendor_id','sdi12_sensor_model','sdi12_sensor_version','sdi12_sensor_sn','pulse_interval','pulse_state','pulse_falling','pulse_rising']]
Data_Type            = ["Soil"]
Data_Types           = ["aux_raw"]

Column_Names         = [Soil_Data_Columns]


for i in range(len(Sensor_ID)):
    Sensor = Sensor_ID[i]
    Habi   = Habitat[i]
    print(Habi)
    Folder_Name_A = str(dirpath + Site[i] + '/' + Habitat[i] + '/' + str(Current_Year) + '/')
    for j in range(len(Data_Type)):
        Folder_Name = str(Folder_Name_A + Data_Type[j] + '/')
        if not os.path.exists(Folder_Name):
            os.makedirs(Folder_Name)
            print "Directory " , Folder_Name ,  " Created "
        else:    
            print "Directory " , Folder_Name ,  " already exists"  
            pass
        write_csv_file(start_current_month, today, Data_Types[j], Data_Type[j], Folder_Name, dt_to_ym(start_current_month), Column_Names[j], Columns_to_Remove[j], Sensor, Habi, Site[i])


2020-04-01 00:00:00
2020-04
Coastal_Sage_Scrub
Directory  C:/Users/ccber-arables/Box/Campus_Lagoon/Coastal_Sage_Scrub/2020/Soil/  already exists
C:/Users/ccber-arables/Box/Campus_Lagoon/Coastal_Sage_Scrub/2020/Soil/Coastal_Sage_Scrub_2020-04_Soil.csv
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.12
0.12857413975
7.09
0.127913068565
7.1
0.1281335173
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.12
0.12857413975
7.11
0.128353874353
7.11
0.128353874353
7.09
0.127913068565
7.12
0.12857413975
7.11
0.128353874353
7.11
0.128353874353
7.09
0.127913068565
7.09
0.127913068565
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.12
0.12857413975
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.11
0.128353874353
7.12
0.12857413975
7.11
0.128353874353
7.11
0.128353874353
7.11
0.1283538743

0.11632275659
6.59
0.11677316707
6.57
0.11632275659
6.59
0.11677316707
6.59
0.11677316707
6.59
0.11677316707
6.55
0.115871973912
6.59
0.11677316707
6.57
0.11632275659
6.58
0.116548008342
6.59
0.11677316707
6.57
0.11632275659
6.59
0.11677316707
6.57
0.11632275659
6.59
0.11677316707
6.53
0.115420818831
6.56
0.116097411789
6.56
0.116097411789
6.55
0.115871973912
6.54
0.115646442935
6.58
0.116548008342
6.53
0.115420818831
6.57
0.11632275659
6.52
0.115195101574
6.52
0.115195101574
6.53
0.115420818831
6.52
0.115195101574
6.52
0.115195101574
6.54
0.115646442935
6.51
0.114969291139
6.52
0.115195101574
6.51
0.114969291139
6.52
0.115195101574
6.52
0.115195101574
6.51
0.114969291139
6.49
0.114517390631
6.52
0.115195101574
6.48
0.114291300506
6.51
0.114969291139
6.5
0.1147433875
6.5
0.1147433875
6.5
0.1147433875
6.45
0.113612470338
6.45
0.113612470338
6.49
0.114517390631
6.47
0.114065117099
6.46
0.113838840385
6.45
0.113612470338
6.45
0.113612470338
6.48
0.114291300506
6.47
0.114065117099
6.44
0.1

10.73
0.202305021873
10.71
0.201927212917
10.68
0.201359881658
10.61
0.200033221918
10.6
0.1998433688
10.62
0.20022299241
10.6
0.1998433688
10.55
0.198892862913
10.53
0.198512081271
10.51
0.198130968299
10.49
0.197749523791
10.48
0.197558677146
10.49
0.197749523791
10.46
0.197176734945
10.45
0.196985639337
10.45
0.196985639337
10.45
0.196985639337
10.38
0.19564564355
10.42
0.196411854178
10.43
0.19660319898
10.38
0.19564564355
10.45
0.196985639337
10.45
0.196985639337
10.45
0.196985639337
10.46
0.197176734945
10.45
0.196985639337
10.51
0.198130968299
10.52
0.198321566214
10.59
0.19965343303
10.67
0.201170606381
10.64
0.200602285619
10.67
0.201170606381
10.68
0.201359881658
Remove location
Remove time
Remove sdi12_version
Remove sdi12_vendor_id
Remove sdi12_sensor_model
Remove sdi12_sensor_version
Remove sdi12_sensor_sn
Remove pulse_interval
Remove pulse_state
Remove pulse_falling
Remove pulse_rising
Successfully wrote C:/Users/ccber-arables/Box/Campus_Lagoon/Coastal_Sage_Scrub/2020/Soi